# MSDS 7331 Data Mining Lab 1
Profesor: Dr. Jake Drew  
Team: Steven Hayden, Josephine MacDaniel, Afreen Siddiqui, & Eduardo Cantu

In [ ]:
import pandas as pd
import numpy as np

## Business Understanding


## Data Meaning Type

## Data Quality

In [5]:
import pandas as pd
import numpy as np

# read in the data from 2015 and 2016

Accident_2015_df = pd.read_csv('https://raw.githubusercontent.com/ecantu75/DataMining_Lab1/master/Data/accident_2015.csv',low_memory=False)
Accident_2016_df = pd.read_csv('https://raw.githubusercontent.com/ecantu75/DataMining_Lab1/master/Data/accident_2016.csv',low_memory=False)
Accident_2016_df = pd.read_csv('https://raw.githubusercontent.com/ecantu75/DataMining_Lab1/master/Data/accident_2016.csv',low_memory=False) 

#combines the 2015 and 2016 file into one data frame
Accident_df = pd.concat([Accident_2016_df,Accident_2015_df])


,Unnamed: 0,state_number,state_name,consecutive_number,number_of_vehicle_forms_submitted_all,number_of_motor_vehicles_in_transport_mvit,number_of_parked_working_vehicles,number_of_forms_submitted_for_persons_not_in_motor_vehicles,number_of_persons_not_in_motor_vehicles_in_transport_mvit,number_of_persons_in_motor_vehicles_in_transport_mvit,...,minute_of_ems_arrival_at_hospital,related_factors_crash_level_1,related_factors_crash_level_1_name,related_factors_crash_level_2,related_factors_crash_level_2_name,related_factors_crash_level_3,related_factors_crash_level_3_name,number_of_fatalities,number_of_drunk_drivers,timestamp_of_crash
0,0,4,Arizona,40484,1,1,0,0,0,1,...,88,0,None,0,None,0,None,1,0,2016-07-16 14:01:00+00:00
1,1,4,Arizona,40615,1,1,0,1,1,1,...,88,0,None,0,None,0,None,1,0,2016-08-28 02:33:00+00:00
2,2,4,Arizona,40610,1,1,0,0,0,3,...,99,0,None,0,None,0,None,2,0,2016-08-16 08:50:00+00:00
3,3,4,Arizona,40140,1,1,0,0,0,1,...,88,0,None,0,None,0,None,1,1,2016-02-13 00:00:00+00:00
4,4,4,Arizona,40606,1,1,0,0,0,2,...,99,0,None,0,None,0,None,1,0,2016-08-09 21:07:00+00:00


## Simple Statistics

## Visualize Attributes

## Explore Joint Attributes

## Explore Attributes and Class

## New Features

## Exceptional Work